<a href="https://colab.research.google.com/github/ElenaVillano/sentiment_analysis_tweets/blob/main/notebooks/limpieza_bien.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTEBOOK DE LIMPIEZA

In [48]:
!pip install "git+https://github.com/ElenaVillano/sentiment_analysis_tweets.git#egg=nlptweet&subdirectory=src" --quiet

In [ ]:
#!pip uninstall nlptweet

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
#import re
import timeit
import string
# Nuestro paquete
import nlp


#call the nltk downloader
nltk.download('punkt')

from dateutil import parser

#from sklearn.model_selection import train_test_split

# Carga un set de stopwords predefinidas
from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
# Nombramiento de columnas
col_names = ['target', # Polaridad del twet 0=negativo, 2=neutral, 4=positivo
             'ids',    # ID tweet
             'date',   # Fecha y hora del tweet
             'flag',   # QUERY
             'user',   # Usuario del tweet
             'text']   # Texto del tweety

# Carga y limpieza de datos

In [51]:
# Requiered to select a file to be imported into colab
# Not useful if running locally
from google.colab import files
uploaded = files.upload()

Saving smaller_sample_200000.csv to smaller_sample_200000 (1).csv
Saving testdata_manual_2009_06_14.csv to testdata_manual_2009_06_14 (1).csv


In [52]:
training =  pd.read_csv('smaller_sample_200000.csv',
                 encoding='latin-1')

In [53]:
test = pd.read_csv('testdata_manual_2009_06_14.csv', names=col_names)

In [54]:
print(training.shape)
print(test.shape)

(200000, 6)
(498, 6)


In [55]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,Sat Jun 20 01:09:32 PDT 2009,NO_QUERY,evilcaz,Far too tired to be at work
8,0,2233840533,Thu Jun 18 21:59:23 PDT 2009,NO_QUERY,TiffanyNicoleN,"I wish today never happened. On top of that, I..."
27,0,2254342354,Sat Jun 20 09:36:05 PDT 2009,NO_QUERY,AJMcCleary,"@ReallyShecky - Indeed, come on down. The sick..."
41,4,1794361968,Thu May 14 05:38:02 PDT 2009,NO_QUERY,cherrybomb54,Now I get to make incredibly long Tweets thank...
44,0,1686244630,Sun May 03 04:31:06 PDT 2009,NO_QUERY,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,Sun Jun 07 07:33:41 PDT 2009,NO_QUERY,heidirenee92,@allucha It would be a good idea not to mess w...
48,4,2004632930,Tue Jun 02 08:30:15 PDT 2009,NO_QUERY,Hadari_B,At a birthday party of a friend !
155,4,1687038778,Sun May 03 07:34:54 PDT 2009,NO_QUERY,lynettebrennan,"is relaxing on the couch, Sundays are the best"


## Convertimos a minúsculas

In [56]:
from nlp.preprocessing import convierte_a_minusculas

In [57]:
training = convierte_a_minusculas(training)
test = convierte_a_minusculas(test)

In [58]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"@reallyshecky - indeed, come on down. the sick..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,@allucha it would be a good idea not to mess w...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Reemplazamos las URLs con el texto "URL"

In [59]:
from nlp.preprocessing import reemplazar_urls

In [60]:
training['text'] = training['text'].map(lambda s: reemplazar_urls(s))
test['text'] = test['text'].map(lambda s: reemplazar_urls(s))

In [61]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"@reallyshecky - indeed, come on down. the sick..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,@allucha it would be a good idea not to mess w...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Reemplazamos las menciones @ con la palabra "USER_MENTION"



In [62]:
from nlp.preprocessing import reemplazar_usuarios

In [63]:
training['text'] = training['text'].map(lambda s: reemplazar_usuarios(s))
test['text'] = test['text'].map(lambda s: reemplazar_usuarios(s))

In [64]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"user_mention - indeed, come on down. the sick ..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Quitamos el Hashtag , pero dejamos la palabra

In [65]:
# The regular expression used to match hashtags is #(\S+).
from nlp.preprocessing import quitar_hashtag

In [66]:
training['text'] = training['text'].map(lambda s: quitar_hashtag(s))
test['text'] = test['text'].map(lambda s: quitar_hashtag(s))

In [67]:
# Ejemplo
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"user_mention - indeed, come on down. the sick ..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Quitamos los Retweets, sólo la palabra RT , dejamos el comentario

In [68]:
#  The regular expression used to match retweets is \brt\b.
from nlp.preprocessing import quitar_RT

In [69]:
training['text'] = training['text'].map(lambda s: quitar_RT(s))
test['text'] = test['text'].map(lambda s: quitar_RT(s))

In [70]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,"i wish today never happened. on top of that, i..."
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,"user_mention - indeed, come on down. the sick ..."
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single? and if i turn...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend !
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,"is relaxing on the couch, sundays are the best"


## Quitamos espacios o puntos extras

In [71]:
## Reemplazamos los dobles puntos (o más) con un espacio , dos o más espacios con 1 espacio y hacemos strip de espacios y comillas
# Strip any punctuation [’"?!,.():;-'] from the word ??
from nlp.preprocessing import quitar_caracteres_especiales

In [72]:
training['text'] = training['text'].map(lambda s: quitar_caracteres_especiales(s))
test['text'] = test['text'].map(lambda s: quitar_caracteres_especiales(s))

In [73]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


## Quitamos expresiones con letras repetidas

In [74]:
#Convert 2 or more letter repetitions to 2 letters.
#Some people send tweets like I am sooooo
#happpppy adding multiple characters to emphasize
#on certain words. This is done to handle such tweets
#by converting them to I am soo happy
from nlp.preprocessing import quitar_letras_repetidas

In [75]:
training['text'] = training['text'].map(lambda s: quitar_letras_repetidas(s))
test['text'] = test['text'].map(lambda s: quitar_letras_repetidas(s))

In [76]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


## Quitar caracteres nonascii

In [77]:
from nlp.preprocessing import quitar_nonascii

In [78]:
training['text'] = training['text'].map(lambda s: quitar_nonascii(s))
test['text'] = test['text'].map(lambda s: quitar_nonascii(s))

In [79]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


## Removemos stopwords

In [83]:
nltk.download('stopwords') # Use predefined list of stopwords from nltk

STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
from nlp.preprocessing import separar_abreviaciones

In [85]:
training['text'] = training['text'].map(lambda s: separar_abreviaciones(s))
test['text'] = test['text'].map(lambda s: separar_abreviaciones(s))

In [86]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far too tired to be at work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,i wish today never happened on top of that i m...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come on down the sick hea...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,now i get to make incredibly long tweets thank...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ being single and if i turn ...
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention it would be a good idea not to me...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,at a birthday party of a friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,is relaxing on the couch sundays are the best


In [87]:
from nlp.preprocessing import remove_stopwords

In [88]:
training['text'] = training['text'].map(lambda s: remove_stopwords(s))
test['atext'] = test['text'].map(lambda s: remove_stopwords(s))

In [89]:
training.loc[[4,8,27,41,44,35,48,155]]

,target,ids,date,flag,user,text
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far tired work
8,0,2233840533,thu jun 18 21:59:23 pdt 2009,no_query,tiffanynicolen,wish today never happened top miss user_mentio...
27,0,2254342354,sat jun 20 09:36:05 pdt 2009,no_query,ajmccleary,user_mention indeed come sick heat really quit...
41,4,1794361968,thu may 14 05:38:02 pdt 2009,no_query,cherrybomb54,get make incredibly long tweets thanks twitzer...
44,0,1686244630,sun may 03 04:31:06 pdt 2009,no_query,anggieholics,btw whats wrong w/ single turn lesbian let u know
35,4,2064953318,sun jun 07 07:33:41 pdt 2009,no_query,heidirenee92,user_mention would good idea mess mr potatohea...
48,4,2004632930,tue jun 02 08:30:15 pdt 2009,no_query,hadari_b,birthday party friend
155,4,1687038778,sun may 03 07:34:54 pdt 2009,no_query,lynettebrennan,relaxing couch sundays best


## Stemming con NLTK

In [91]:
from nlp.preprocessing import oracion_raiz

In [92]:
training['text'] = training['text'].map(lambda s: oracion_raiz(s))
test['atext'] = test['text'].map(lambda s: oracion_raiz(s))

In [93]:
training

,target,ids,date,flag,user,text
0,0,2297439235,tue jun 23 10:26:35 pdt 2009,no_query,lizamtl,user_ment wellnot breakup speech per sebut go ...
1,4,2057851146,sat jun 06 13:47:00 pdt 2009,no_query,maiseyjon,state slow seem pick bit
2,0,1686056224,sun may 03 03:31:34 pdt 2009,no_query,tynnn,user_ment hey help user_ment take url behind a...
3,0,2005592004,tue jun 02 09:55:03 pdt 2009,no_query,michaelmidnight,aw man user_ment think make see chang thing wo...
4,0,2250671573,sat jun 20 01:09:32 pdt 2009,no_query,evilcaz,far tire work
...,...,...,...,...,...,...
199995,0,2257780639,sat jun 20 14:52:59 pdt 2009,no_query,kateybella,user_ment think kati perri remind situat kirk
199996,0,2229302468,thu jun 18 15:32:54 pdt 2009,no_query,sexi_lexi524,would bf build someth w/n would much would get...
199997,0,1999605224,mon jun 01 20:30:30 pdt 2009,no_query,azndelicious,tip finger hurt nervou school store tomorrow p...
199998,4,1553926326,sat apr 18 15:35:25 pdt 2009,no_query,erikahoney,url chillin pool


Quitamos datos al entrenamiento, empecemos con 100,000 

In [ ]:
data_set_1 = training= training.head(100000)

In [ ]:
data_set_1

,target,ids,date,flag,user,text
0,0,1467810369,mon apr 06 22:19:45 pdt 2009,no_query,_thespecialone_,user_ment url aww bummer shoulda got david car...
1,0,1467810672,mon apr 06 22:19:49 pdt 2009,no_query,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,mon apr 06 22:19:53 pdt 2009,no_query,mattycus,user_ment dive mani time ball manag save 50 % ...
3,0,1467811184,mon apr 06 22:19:57 pdt 2009,no_query,ellectf,whole bodi feel itchi like fire
4,0,1467811193,mon apr 06 22:19:57 pdt 2009,no_query,karoli,user_ment behav mad see
...,...,...,...,...,...,...
99995,0,1793820630,thu may 14 04:05:58 pdt 2009,no_query,babybigmouth,son develop new habit wake 530am second pot co...
99996,0,1793820812,thu may 14 04:06:00 pdt 2009,no_query,paramoreroxx,look like router broke tweet fone
99997,0,1793820858,thu may 14 04:06:00 pdt 2009,no_query,beckypope,realli dont want colleg right wish sunni
99998,0,1793820982,thu may 14 04:06:02 pdt 2009,no_query,xmissxxxmurderx,user_ment *offer pepto*
